# 선형 회귀식의 계수를 찾는 법 - OLS VS. SGD
- 보스턴 집값 데이터 활용(RM VS Price)

### 필요한 모듈 import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 필요한 라이브러리 import 


### 데이터 수집 및 분할

In [2]:
from sklearn import datasets

boston = datasets.fetch_openml('boston')
boston

C:\Users\peter\AppData\Roaming\Python\Python39\site-packages\sklearn\datasets\_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
C:\Users\peter\AppData\Roaming\Python\Python39\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


{'data':         CRIM    ZN  INDUS CHAS    NOX     RM   AGE     DIS RAD    TAX  \
 0    0.00632  18.0   2.31    0  0.538  6.575  65.2  4.0900   1  296.0   
 1    0.02731   0.0   7.07    0  0.469  6.421  78.9  4.9671   2  242.0   
 2    0.02729   0.0   7.07    0  0.469  7.185  61.1  4.9671   2  242.0   
 3    0.03237   0.0   2.18    0  0.458  6.998  45.8  6.0622   3  222.0   
 4    0.06905   0.0   2.18    0  0.458  7.147  54.2  6.0622   3  222.0   
 ..       ...   ...    ...  ...    ...    ...   ...     ...  ..    ...   
 501  0.06263   0.0  11.93    0  0.573  6.593  69.1  2.4786   1  273.0   
 502  0.04527   0.0  11.93    0  0.573  6.120  76.7  2.2875   1  273.0   
 503  0.06076   0.0  11.93    0  0.573  6.976  91.0  2.1675   1  273.0   
 504  0.10959   0.0  11.93    0  0.573  6.794  89.3  2.3889   1  273.0   
 505  0.04741   0.0  11.93    0  0.573  6.030  80.8  2.5050   1  273.0   
 
      PTRATIO       B  LSTAT  
 0       15.3  396.90   4.98  
 1       17.8  396.90   9.14  
 2       

In [7]:
type(X.RM)

pandas.core.series.Series

In [9]:
np.array(boston.data.RM).reshape(-1, 1)

array([[6.575],
       [6.421],
       [7.185],
       [6.998],
       [7.147],
       [6.43 ],
       [6.012],
       [6.172],
       [5.631],
       [6.004],
       [6.377],
       [6.009],
       [5.889],
       [5.949],
       [6.096],
       [5.834],
       [5.935],
       [5.99 ],
       [5.456],
       [5.727],
       [5.57 ],
       [5.965],
       [6.142],
       [5.813],
       [5.924],
       [5.599],
       [5.813],
       [6.047],
       [6.495],
       [6.674],
       [5.713],
       [6.072],
       [5.95 ],
       [5.701],
       [6.096],
       [5.933],
       [5.841],
       [5.85 ],
       [5.966],
       [6.595],
       [7.024],
       [6.77 ],
       [6.169],
       [6.211],
       [6.069],
       [5.682],
       [5.786],
       [6.03 ],
       [5.399],
       [5.602],
       [5.963],
       [6.115],
       [6.511],
       [5.998],
       [5.888],
       [7.249],
       [6.383],
       [6.816],
       [6.145],
       [5.927],
       [5.741],
       [5.966],
       [

In [12]:
X = np.array(boston.data.RM).reshape(-1, 1)
y = boston.target

# 데이터 분할

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)

# 1. LinearRegression 모델을 사용한 경우

In [17]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg = reg.fit(X_train, y_train)

print(reg.coef_, reg.intercept_)

y_pred = reg.predict(X_test)

from eval_score import evaluate_score
evaluate_score(y_test, y_pred)

[8.46109164] -30.571032410898336
mse: 36.517
mrse: 6.043
r2: 0.602


# 2. SGDRegressor with hyperparameter

In [30]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(max_iter=100_000_000, eta0=0.0001, learning_rate='invscaling',
                   loss='squared_error', random_state=42)
reg.fit(X_train, y_train)

print(reg.coef_[0], reg.intercept_[0])

3.5471540937480923 0.497708365257496


In [25]:
y_pred = reg.predict(X_test)
evaluate_score(y_test, y_pred)

mse: 55.130
mrse: 7.425
r2: 0.399


# 3. SGDRegressor with scaling

In [35]:
# 표준화 스케일링을 사용하여 경사하강법 경사하강법 모델링

# 스케일링 데이터 준비
train_mean = np.mean(X_train, axis=0)
train_std = np.std(X_train, axis=0)

X_train_scaled = (X_train - train_mean) / train_std
X_test_scaled = (X_test - train_mean) / train_std

# 모델 객체 생성
reg = SGDRegressor(max_iter=100_000_000, eta0=0.0001, learning_rate='invscaling',
                   loss='squared_error', random_state=42)
reg.fit(X_train_scaled, y_train)

# 계수 확인
print(reg.coef_, reg.intercept_)

# 평가 지표
y_pred = reg.predict(X_test_scaled)

evaluate_score(y_test, y_pred)

[5.62395551] [21.47086473]
mse: 37.668
mrse: 6.137
r2: 0.589


# 4. SGDRegressor with StandardScaler()

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
# X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[ 8.61041758e-01],
       [ 6.39842106e-01],
       [-3.23026964e-01],
       [-5.42780866e-01],
       [-1.28878753e+00],
       [ 1.71419315e-01],
       [ 5.44422649e-01],
       [-2.62305491e-01],
       [-4.37241163e-01],
       [ 2.36478036e-01],
       [ 7.17912572e-01],
       [ 8.97185491e-01],
       [-7.55306021e-01],
       [-6.74344057e-01],
       [-3.33147209e-01],
       [-9.33133192e-01],
       [ 5.71891887e-01],
       [ 3.57326755e+00],
       [-1.32188049e-01],
       [-1.91913425e+00],
       [-7.92895504e-01],
       [-9.04218204e-01],
       [ 4.53340440e-01],
       [ 2.33586538e-01],
       [ 5.28678684e-02],
       [-5.16757377e-01],
       [-2.27607506e-01],
       [-3.46158954e-01],
       [ 1.83113957e+00],
       [-1.24830655e+00],
       [ 4.70689432e-01],
       [-9.37470440e-01],
       [ 2.57425474e+00],
       [-3.51635648e-03],
       [-1.58211537e-01],
       [-4.73384897e-01],
       [ 1.26601085e-01],
       [ 1.29187507e+00],
       [-3.0

# 5. Pipeline

In [42]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

reg = make_pipeline(StandardScaler(),
                    SGDRegressor(max_iter=1000000, eta0=0.01,\
                                 tol=0.0001, random_state=42, loss='squared_error'))
reg.fit(X_train, y_train)

# 계수 및 절편 확인: _속성은 학습을 통해 결정되는 속성
print(reg[1].coef_, reg[1].intercept_)

#회귀식 - pipeline()을 사용했기 때문에 SGDRegressor의 parameter가 reg객체의 1번 인덱스에 들어감
print("y = {:2f}X + {:.3f}".format(reg[1].coef_[0], reg[1].intercept_[0]))

# 예측 수행
y_pred = reg.predict(X_test)
from sklearn.metrics import mean_squared_error, r2_score
# MSE, RMSE, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MSE:", np.round(mse, 3))
print("RMSE: ", np.round(rmse, 3))
print("R2: ", np.round(r2, 3))

[5.84750366] [22.31897879]
y = 5.847504X + 22.319
MSE: 36.523
RMSE:  6.043
R2:  0.602
